In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [2]:
def read_csv(file):
    df = pd.read_csv(file, parse_dates=True, index_col=0)
    return df

In [3]:
def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

## XGBoost Without PCA and without Feature Engineering

In [4]:
files = ['CompleteWeeklyIndexes.csv','MeanWeeklyImputed.csv', 'KNNWeeklyImputed.csv', 'MovingAverageWeeklyImputed.csv',  'RegressionWeeklyImputed.csv', 'MIIWeeklyImputed.csv']

for file in files:
    df = read_csv(file)

    print(file)
    if "Complete" in file:
        X_test = read_csv('CompleteWeeklyIndexes.csv')
    else:
        test_data = file.split('.')
        X_test = read_csv(test_data[0] + '_test.' + test_data[1])
        
    y = pd.read_csv('sp500_target_regimes.csv', parse_dates=True)
    y.index = y['date'].values
    y = y[['regime']]
    y = y['regime']=='BEAR'
    y = pd.DataFrame (y, columns = ['regime'])
       

    X_train = df.loc['2000-01-01':'2015-01-01']
    X_test = X_test.loc['2015-01-02':]
    y_train = y.loc['2000-01-01':'2015-01-01']
    y_test = y.loc['2015-01-02':]
    
    dtrain = xgb.DMatrix(X_train, label=y_train)
    
    param_test1 = {
        'max_depth':range(2,6,1),
        'min_child_weight':range(1,6,1),
        'gamma':np.arange(0.1,0.5,0.1),
        'max_delta_step':range(1,10,1),
    }
    '''
    balance = np.sqrt(len(y_train[y_train['regime']==False])/len(y_train[y_train['regime']==True]))
    gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier(objective="binary:logistic",
                                                          disable_default_eval_metric = 1,
                                                          learning_rate=0.1, 
                                                          n_estimators=200, 
                                                          random_state=27, 
                                                          min_child_weight=2, 
                                                          gamma=2, 
                                                          max_depth=2,
                                                          max_delta_step=1,
                                                          scale_pos_weight=balance), 
                            param_grid = param_test1, 
                            scoring='f1',
                            n_jobs=4,
                            iid=False, 
                            cv=5)

    gsearch1.fit(X_train, y_train, eval_metric=f1_eval)
    print(gsearch1.best_params_, gsearch1.best_score_)
    '''
    xgb_model = xgb.XGBClassifier(objective="binary:logistic",
                                  disable_default_eval_metric = 1,
                                  #learning_rate=0.1, 
                                  n_estimators=500, 
                                  random_state=27, 
                                  #min_child_weight=gsearch1.best_params_['min_child_weight'], 
                                  #max_delta_step=gsearch1.best_params_['max_delta_step'],
                                  #gamma=gsearch1.best_params_['gamma'], 
                                  #max_depth=gsearch1.best_params_['max_depth'], 
                                  #scale_pos_weight=balance
                                 )

    xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric=f1_eval, verbose=False, early_stopping_rounds=100)

    y_train_pred = xgb_model.predict(X_train, ntree_limit=xgb_model.best_ntree_limit)
    print("____________Training Data______________")
    tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("F1 score", 1-f_score)
    print("---------------------------------------")
    
    print("______________Test Data________________")
    y_pred = xgb_model.predict(X_test, ntree_limit=xgb_model.best_ntree_limit)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f_score = 2*precision*recall/(precision+recall)  
    print("    ","True", "False")
    print("True ", " ", tp, "  ", fp)
    print("False", " ",fn,"  ", tn)
    print("_______________________________________")
    print("F1 score", 1-f_score)
    print("---------------------------------------")


CompleteWeeklyIndexes.csv
____________Training Data______________
     True False
True    370    0
False   2    410
_______________________________________
F1 score 0.0026954177897574594
---------------------------------------
______________Test Data________________
     True False
True    39    11
False   26    196
_______________________________________
F1 score 0.32173913043478264
---------------------------------------
MeanWeeklyImputed.csv
____________Training Data______________
     True False
True    370    0
False   2    410
_______________________________________
F1 score 0.0026954177897574594
---------------------------------------
______________Test Data________________
     True False
True    45    23
False   20    184
_______________________________________
F1 score 0.32330827067669177
---------------------------------------
KNNWeeklyImputed.csv
____________Training Data______________
     True False
True    372    2
False   0    408
_______________________________________